In [2]:
import sys; sys.path.append('../..')
from abslithist import *
from abslithist.realism.passages import *

In [3]:
tokenize_agnostic("This is a test.\nHello?")

['This', ' ', 'is', ' ', 'a', ' ', 'test', '.', '\n', 'Hello', '?']

In [6]:
def to_sentdf(txt,words_recog=set(),num_word_min=45,vald={},valname='val',para_split='\n\n'):
    ntok,nword,npara,nsent=0,-1,0,0
    txt=txt.strip()
    o=[]
    
    def _cleantok(x):
        return {
            ' ':'_',
            '\n':'|'
        }.get(x,x)
    
    for pi,para in enumerate(txt.split(para_split)):
        para=para.strip()
        for si,sent in enumerate(tokenize_sentences(para)):
            sent=sent.strip()
            swords=tokenize_agnostic(sent)
            swords=[_cleantok(x) for x in swords]
            for w in swords:
                w=w.strip()
                wl=w.lower()
                ispunc=int(not w or not w[0].isalpha())
                if not ispunc: nword+=1
                dx={
                    'i_para':npara,
                    'i_sent':nsent,
                    'i_word':nword,# if not ispunc else None,
                    'i_tok':ntok,
                    'tok':w,
                    'tokl':wl,
                    'is_punct':ispunc,
                }
                if words_recog: dx['is_recog']=int((w in words_recog) or (wl in words_recog))
                if vald: dx[valname]=vald.get(wl,np.nan)
                o+=[dx]
                ntok+=1
            nsent+=1
        npara+=1
    odf=pd.DataFrame(o)
    return odf.set_index('i_tok') if 'i_tok' in odf.columns else odf

# Test
sentdf=to_sentdf(willoughby_full,para_split='\n\n')
sentdf

,i_para,i_sent,i_word,tok,tokl,is_punct
i_tok,,,,,,
0,0,0,0,A,a,0
1,0,0,0,_,_,1
2,0,0,1,gentleman,gentleman,0
3,0,0,1,_,_,1
4,0,0,2,carrying,carrying,0
...,...,...,...,...,...,...
1150,3,18,532,_,_,1
1151,3,18,533,was,was,0
1152,3,18,533,_,_,1


## Pasage df

In [9]:
def to_psgdf(sentdf_or_txt,tfield='Abs-Conc.Median.median',norms=None,tokname='tokl',valname='val',nmin=50):
    if norms is None: norms=get_allnorms()
    normsok=norms[tfield].dropna()
    w2score=dict(zip(normsok.index, normsok))
    wordsok=set(normsok.index)
    sentdf = to_sentdf(sentdf_or_txt,para_split='\n\n\n\n\n') if type(sentdf_or_txt)==str else sentdf_or_txt
    if tokname in sentdf.columns:
        sentdf['is_recog']=sentdf[tokname].apply(lambda w: int(w in wordsok))
        sentdf[valname]=sentdf[tokname].apply(lambda w: w2score.get(w))
    return sentdf#.set_index('i_tok')

# Test
psgdf=to_psgdf(willoughby_full)
psgdf

,i_para,i_sent,i_word,tok,tokl,is_punct,is_recog,val
i_tok,,,,,,,,
0,0,0,0,A,a,0,0,NaN
1,0,0,0,_,_,1,0,NaN
2,0,0,1,gentleman,gentleman,0,1,-0.081184
3,0,0,1,_,_,1,0,NaN
4,0,0,2,carrying,carrying,0,1,0.404267
...,...,...,...,...,...,...,...,...
1150,0,18,532,_,_,1,0,NaN
1151,0,18,533,was,was,0,0,NaN
1152,0,18,533,_,_,1,0,NaN


## Scaling up

In [10]:
# func
def do_text(inp):
    path,ofn,tfield=inp
    if not os.path.exists(path): return
    odir=os.path.dirname(ofn)
    if not os.path.exists(odir): os.makedirs(odir)
    
    # load
    with open(path) as f: txt=f.read()
    psgdf=to_psgdf(txt,tfield=tfield)
    psgdf.reset_index().to_feather(ofn)
    del psgdf
#     return psgdf
#     psgdf.to_csv(ofn,index=False)

# objs
objs = [
    (t.path_txt,
     f'../../data/scores/bytext3/{t.corpus.name}/{t.id}.ft',
     f'Abs-Conc.Median.{to_field_period(t.year)}'
    ) for t in C.texts()
]
objs = [inp for inp in objs if not os.path.exists(inp[1])]

# Test
do_text(objs[0])

In [11]:
# Do all
res=pmap(
    do_text,
    objs,
    num_proc=4,
    desc='Scoring passages'
)

Scoring passages [x4]: 100%|██████████| 1809/1809 [25:00<00:00,  1.21it/s]
